# Simple Index Demo + ChatGPT

Use a very simple wrapper around the ChatGPT API

#### Load documents, build the GPTSimpleVectorIndex

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader, LLMPredictor
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display

In [2]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [ ]:
index = GPTSimpleVectorIndex(documents, chunk_size_limit=512)

In [4]:
# LLM Predictor (gpt-3.5-turbo)
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

#### Query Index

By default, with the help of langchain's PromptSelector abstraction, we use 
a modified refine prompt tailored for ChatGPT-use if the ChatGPT model is used.

In [ ]:
response = index.query(
    "What did the author do growing up?", 
    llm_predictor=llm_predictor,
    similarity_top_k=3
)

In [6]:
display(Markdown(f"<b>{response}</b>"))

<b>Before college, the author worked on writing essays and programming. They wrote short stories and essays on various topics. They also tried programming on an IBM 1401 in 9th grade using an early version of Fortran. In college, the author studied painting and art history and spent a year in Florence, Italy, where they painted portraits of people and still life. After college, the author worked on software development and co-founded a startup called Viaweb. Later, the author spent three months writing essays in 2015 before returning to work on Bel, a programming language they had been developing for years. The author worked intensively on Bel, often having chunks of the code in their head and working on it while watching their children play. Most of Bel was written while the author was living in England, where they moved with their family in 2016. In the fall of 2019, Bel was finally finished, and the author resumed writing essays and thinking about other projects.</b>

In [ ]:
response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor,
    similarity_top_k=5
)

In [9]:
display(Markdown(f"<b>{response}</b>"))

<b>The author attended RISD to learn how to paint and took a color class there. However, he mostly taught himself to paint and dropped out in 1993. He then moved to New York City to pursue his career as an artist.</b>

**Refine Prompt**: Here is the chat refine prompt 

In [11]:
from gpt_index.prompts.chat_prompts import CHAT_REFINE_PROMPT

In [22]:
dict(CHAT_REFINE_PROMPT.prompt)

{'input_variables': ['context_msg', 'query_str', 'existing_answer'],
 'output_parser': None,
 'partial_variables': {},
 'messages': [HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query_str'], output_parser=None, partial_variables={}, template='{query_str}', template_format='f-string', validate_template=True), additional_kwargs={}),
  AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['existing_answer'], output_parser=None, partial_variables={}, template='{existing_answer}', template_format='f-string', validate_template=True), additional_kwargs={}),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context_msg'], output_parser=None, partial_variables={}, template="We have the opportunity to refine the above answer (only if needed) with some more context below.\n------------\n{context_msg}\n------------\nGiven the new context, refine the original answer to better answer the question. If the context isn't useful, output the original answ

#### Query Index (Using the standard Refine Prompt)

If we use the "standard" refine prompt (where the prompt is one text template instead of multiple messages), we find that the results over ChatGPT are worse. 

In [27]:
from gpt_index.prompts.default_prompts import DEFAULT_REFINE_PROMPT

In [ ]:
response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor,
    refine_template=DEFAULT_REFINE_PROMPT,
    similarity_top_k=5
)

In [31]:
display(Markdown(f"<b>{response}</b>"))

<b>

The existing answer is not relevant to the new context provided and therefore the original answer remains sufficient. The author dropped out of RISD in 1993 and moved to New York to pursue painting.</b>

### [Beta] Use ChatGPTLLMPredictor

Very simple GPT-Index-native ChatGPT wrapper. Note: this is a beta feature. If this doesn't work please
use the suggested flow above.

In [3]:
# use ChatGPT [beta]
from gpt_index.langchain_helpers.chatgpt import ChatGPTLLMPredictor

llm_predictor = ChatGPTLLMPredictor()

In [ ]:
response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor
)